In [3]:
!rm -rf clone && git clone https://github.com/rusgu-real/ecg_digitisation clone && cp -a clone/. .

Cloning into 'clone'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 36 (delta 6), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 24.34 MiB | 9.56 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

In [ ]:


# --- Chargement image ---
img = cv2.imread("data/test/145375843-0009.png")  # <-- mets ton chemin ici
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# --- Binarisation adaptative ---
bin_img = cv2.adaptiveThreshold(
    gray,
    255,
    cv2.ADAPTIVE_THRESH_MEAN_C,
    cv2.THRESH_BINARY_INV,
    blockSize=31,
    C=5
)

# --- Extraction des lignes horizontales ---
horiz_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50, 1))
horiz = cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, horiz_kernel)

# --- Extraction des lignes verticales ---
vert_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 50))
vert = cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, vert_kernel)

# --- Grille finale ---
grid = cv2.bitwise_or(horiz, vert)

# --- Affichage ---
plt.figure(figsize=(10, 10))
plt.imshow(grid, cmap="gray")
plt.axis("off")
plt.title("Grille détectée (courbe supprimée)")
plt.show()


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# --- 1. Load image and resize ---
img = cv2.imread("data/test/145375843-0009.png")
if img is None:
    raise IOError("Image not found")

img = cv2.resize(img, None, fx=0.7, fy=0.7)
h_img, w_img = img.shape[:2]

cv2_imshow(img)
print("Original Image shape:", img.shape)

# --- 2. Convert to HSV and create lenient red mask ---
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
lower_red1 = np.array([0, 50, 50])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([170, 50, 50])
upper_red2 = np.array([180, 255, 255])

mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

# Minimal morphology to remove tiny noise
kernel = np.ones((2, 2), np.uint8)
red_mask_clean = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, kernel, iterations=1)

cv2_imshow(red_mask_clean)
print("Red mask cleaned")

# --- 3. Edge detection ---
edges = cv2.Canny(red_mask_clean, 20, 60)
cv2_imshow(edges)
print("Edges detected")

# --- 4. Hough line detection (lenient) ---
lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=20, minLineLength=20, maxLineGap=15)
grid_img = img.copy()
horizontal_lines = []
vertical_lines = []

if lines is not None:
    for x1, y1, x2, y2 in lines[:,0]:
        if abs(y1 - y2) < 5:
            horizontal_lines.append((x1, y1, x2, y2))
            cv2.line(grid_img, (x1, y1), (x2, y2), (0,255,0), 1)
        elif abs(x1 - x2) < 5:
            vertical_lines.append((x1, y1, x2, y2))
            cv2.line(grid_img, (x1, y1), (x2, y2), (0,255,0), 1)

cv2_imshow(grid_img)
print("Grid detection done: {} horizontal, {} vertical lines".format(len(horizontal_lines), len(vertical_lines)))

# --- 5. Reconstruct grid ---
def cluster_positions(positions, tolerance=3):
    positions = sorted(positions)
    clusters = []
    current_cluster = [positions[0]]
    for pos in positions[1:]:
        if abs(pos - current_cluster[-1]) <= tolerance:
            current_cluster.append(pos)
        else:
            clusters.append(int(np.mean(current_cluster)))
            current_cluster = [pos]
    clusters.append(int(np.mean(current_cluster)))
    return clusters

h_positions = [line[1] for line in horizontal_lines]
v_positions = [line[0] for line in vertical_lines]

h_lines_unique = cluster_positions(h_positions)
v_lines_unique = cluster_positions(v_positions)

# Overlay reconstructed grid
grid_reconstructed = img.copy()
for y in h_lines_unique:
    cv2.line(grid_reconstructed, (0, y), (w_img, y), (255,0,0), 1)
for x in v_lines_unique:
    cv2.line(grid_reconstructed, (x,0), (x,h_img), (255,0,0), 1)

cv2_imshow(grid_reconstructed)
print("Grid reconstructed: {} horizontal, {} vertical lines".format(len(h_lines_unique), len(v_lines_unique)))

# --- 6. Extract ECG waveforms ---
# Create pixel-to-voltage mapping
v_spacing_px = int(np.median(np.diff(h_lines_unique)))
h_spacing_px = int(np.median(np.diff(v_lines_unique)))

# Split vertical grid into 4 regions (assuming 4 ECG traces stacked)
num_traces = 4
trace_height = h_img // num_traces

ecg_traces = []

for i in range(num_traces):
    y_start = i * trace_height
    y_end = (i+1) * trace_height
    ecg_crop = img[y_start:y_end, :, :]

    # Convert to grayscale and invert (ECG is darker than background)
    gray = cv2.cvtColor(ecg_crop, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)

    # Threshold to get ECG line
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)

    # Get waveform as pixel coordinates
    xs = np.arange(thresh.shape[1])
    ys = []
    for x in xs:
        column = thresh[:, x]
        if np.any(column > 0):
            y_pos = np.argmax(column > 0)
            ys.append(y_pos)
        else:
            ys.append(np.nan)  # missing data
    ecg_traces.append((xs, np.array(ys)))

    # Plot waveform
    plt.figure(figsize=(12,2))
    plt.plot(xs, ys)
    plt.title(f"ECG Trace {i+1}")
    plt.gca().invert_yaxis()  # match image coordinates
    plt.show()

print("ECG extraction done")


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# --- Step 0: Load and resize image ---
img = cv2.imread("data/test/145375843-0009.png")
if img is None:
    raise IOError("Image not found")
img = cv2.resize(img, None, fx=0.7, fy=0.7)
h_img, w_img = img.shape[:2]

cv2_imshow(img)
print("Step 0: Original Image")

# --- Step 1: Detect red grid mask ---
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
lower_red1 = np.array([0,50,50])
upper_red1 = np.array([10,255,255])
lower_red2 = np.array([170,50,50])
upper_red2 = np.array([180,255,255])
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

# Minimal morphology to clean noise
kernel = np.ones((2,2),np.uint8)
red_mask_clean = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, kernel, iterations=1)

cv2_imshow(red_mask_clean)
print("Step 1: Red grid mask cleaned")

# --- Step 2: Edge detection ---
edges = cv2.Canny(red_mask_clean, 20, 60)
cv2_imshow(edges)
print("Step 2: Edges detected")

# --- Step 3: Detect Hough lines ---
lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=20, minLineLength=20, maxLineGap=15)
grid_detected = img.copy()
horizontal_lines = []
vertical_lines = []
if lines is not None:
    for x1, y1, x2, y2 in lines[:,0]:
        if abs(y1 - y2) < 5:
            horizontal_lines.append((x1, y1, x2, y2))
            cv2.line(grid_detected, (x1,y1),(x2,y2),(0,255,0),1)
        elif abs(x1 - x2) < 5:
            vertical_lines.append((x1, y1, x2, y2))
            cv2.line(grid_detected, (x1,y1),(x2,y2),(0,255,0),1)

cv2_imshow(grid_detected)
print(f"Step 3: Grid detected - {len(horizontal_lines)} horizontal, {len(vertical_lines)} vertical lines")

# --- Step 4: Reconstruct grid ---
def cluster_positions(positions, tolerance=3):
    positions = sorted(positions)
    clusters = []
    current_cluster = [positions[0]]
    for pos in positions[1:]:
        if abs(pos - current_cluster[-1]) <= tolerance:
            current_cluster.append(pos)
        else:
            clusters.append(int(np.mean(current_cluster)))
            current_cluster = [pos]
    clusters.append(int(np.mean(current_cluster)))
    return clusters

h_lines_unique = cluster_positions([line[1] for line in horizontal_lines])
v_lines_unique = cluster_positions([line[0] for line in vertical_lines])

grid_reconstructed = img.copy()
for y in h_lines_unique:
    cv2.line(grid_reconstructed, (0,y),(w_img,y),(255,0,0),1)
for x in v_lines_unique:
    cv2.line(grid_reconstructed, (x,0),(x,h_img),(255,0,0),1)

cv2_imshow(grid_reconstructed)
print(f"Step 4: Grid reconstructed - {len(h_lines_unique)} horizontal, {len(v_lines_unique)} vertical lines")

5# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Remove red grid by using the red mask
# Any pixel that is part of the red grid -> set to 255 (white)
gray_no_grid = gray.copy()
gray_no_grid[red_mask_clean > 0] = 255

cv2_imshow(gray_no_grid)
print("Step 5a: Grayscale with red grid removed")

# Now threshold to detect ECG line (black line -> white)
# Adaptive threshold is robust to lighting
thresh_ecg = cv2.adaptiveThreshold(
    gray_no_grid,
    255,
    cv2.ADAPTIVE_THRESH_MEAN_C,
    cv2.THRESH_BINARY_INV,
    blockSize=15,
    C=10
)

threshold_value = 50  # adjust depending on your ECG darkness
thresh_ecg = np.zeros_like(gray_no_grid, dtype=np.uint8)
thresh_ecg[gray_no_grid < threshold_value] = 255  # ECG lines become white


cv2_imshow(thresh_ecg)
print("Step 5b: Thresholded ECG lines (grid removed)")

# Find contours of ECG traces
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter and sort contours by vertical position
ecg_traces = []
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    if w > w_img//20:  # skip small noise
        ecg_traces.append((x, y, w, h))
ecg_traces = sorted(ecg_traces, key=lambda t: t[1])

# Step 5b: Extract pixel coordinates and plot each trace
for i, (x, y, w, h) in enumerate(ecg_traces):
    crop = thresh[y:y+h, x:x+w]
    xs = np.arange(crop.shape[1])
    ys = []
    for col in range(crop.shape[1]):
        column = crop[:, col]
        if np.any(column > 0):
            y_pos = np.argmax(column > 0)
            ys.append(y_pos)
        else:
            ys.append(np.nan)
    ys = np.array(ys)
    plt.figure(figsize=(12,2))
    plt.plot(xs, ys)
    plt.gca().invert_yaxis()
    plt.title(f"Step 5b: ECG Trace {i+1}")
    plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from abc import ABC, abstractmethod

class ImageData:
    def __init__(self, image, metadata=None):
        self.image = image
        self.metadata = metadata

class Processor(ABC):
  @abstractmethod
  def process(self, data:ImageData) -> ImageData:
    pass

class Crop(Processor):
  pass

class GridDetect(Processor):
  pass

class CurveDetect(Processor):
  pass

class TreatImage(Processor):
  def __init__(self,processors : list[Processor]) -> None:
    self.processors = processors

  def run(self, image) -> dict:
    data = ImageData(image, {})
    for processor in self.processors:
      data = processor.process(data)

'''
treat = TreatImage([Crop(),GridDetect(),CurveDetect()])
treat.run(img)
'''
